**Definição das funções que baixam o vídeo do youtube, extraem seu áudio e cuidam para que o título do vídeo não tenha caracteres impróprios para a criação da pasta**

In [21]:
import yt_dlp
import os
import re

def sanitize_filename(title):
    """Remove caracteres inválidos para nomes de arquivo"""
    return re.sub(r'[\\/*?:"<>|]', "_", title)

def download_audio(url, temp_dir):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(temp_dir, '%(title)s.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }],
        'quiet': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=True)
        title = sanitize_filename(info_dict['title'])
        return title, ydl.prepare_filename(info_dict).replace('.webm', '.wav').replace('.m4a', '.wav')

**Trecho do código que propriamente extrai o áudio baixado em 6 faixas.**

In [ ]:
from demucs.separate import main as demucs_separate
import shutil

def extract_and_organize_tracks(input_wav, output_dir):
    temp_process_dir = os.path.join(output_dir, "temp_demucs")
    
    demucs_args = [
        "-n", "htdemucs_6s",
        "--mp3",
        "--float32",
        "-o", temp_process_dir,
        input_wav
    ]
    demucs_separate(demucs_args)
    
    base_name = os.path.splitext(os.path.basename(input_wav))[0]
    source_dir = os.path.join(temp_process_dir, "htdemucs_6s", base_name)
    target_dir = os.path.join(output_dir, "tracks")
    
    os.makedirs(target_dir, exist_ok=True)
    for file in os.listdir(source_dir):
        shutil.move(
            os.path.join(source_dir, file),
            os.path.join(target_dir, file)
        )
    
    shutil.rmtree(temp_process_dir)

In [ ]:
def main(url):
    temp_dir = "temp"
    base_output_dir = "output"
    os.makedirs(temp_dir, exist_ok=True)
    os.makedirs(base_output_dir, exist_ok=True)

    try:
        print("⏬ Baixando áudio do YouTube...")
        title, audio_file = download_audio(url, temp_dir)
        
        video_output_dir = os.path.join(base_output_dir, title)
        os.makedirs(video_output_dir, exist_ok=True)
        
        print("🎸 Extraindo guitarra com IA... (isso pode demorar)")
        extract_and_organize_tracks(audio_file, video_output_dir)
        
        guitar_path = os.path.join(video_output_dir, "guitar.mp3")
        
        print(f"\n✅ Extração concluída! Guitarra disponível em:\n{guitar_path}")

    except Exception as e:
        print(f"\n❌ Erro: {e}")
    finally:
        if os.path.exists(audio_file):
            os.remove(audio_file)

if __name__ == "__main__":
    youtube_url = input("Cole a URL do YouTube: ")
    main(youtube_url)

⏬ Baixando áudio do YouTube...
🎸 Extraindo guitarra com IA... (isso pode demorar)      
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in D:\tab_creator\output\Avenged Sevenfold - Shepherd Of Fire [Official Music Video]\temp_demucs\htdemucs_6s
Separating track temp\Avenged Sevenfold - Shepherd Of Fire [Official Music Video].wav


100%|██████████████████████████████████████████████| 327.59999999999997/327.59999999999997 [02:23<00:00,  2.29seconds/s]



✅ Extração concluída! Guitarra disponível em:
output\Avenged Sevenfold - Shepherd Of Fire [Official Music Video]\guitar.mp3
